<a href="https://colab.research.google.com/github/A-Akhlaghi246/PYTHON/blob/main/fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hazm
!pip install gensim
!pip install python-bidi
!pip install tensorflow
!pip install keras
!pip install --upgrade numpy pandas
!pip install numpy==1.26.4 pandas==2.1.4
!pip install --no-binary :all: pandas
!pip install --upgrade pandas pandasai

SyntaxError: invalid syntax (ipython-input-2-780005940.py, line 1)

In [ ]:
import pandas as pd
import numpy as np
import re
from hazm import Normalizer, word_tokenize, Stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
from bidi.algorithm import get_display
import arabic_reshaper
import warnings
warnings.filterwarnings('ignore')

class PersianFakeNewsDetector:
    def __init__(self):
        self.vectorizer = None
        self.model = None
        self.word2vec_model = None
        self.normalizer = Normalizer()
        self.stemmer = Stemmer()

    def load_dataset(self, file_path):
        """بارگذاری دیتاست از فایل CSV"""
        try:
            df = pd.read_csv(file_path)
            print(f"دیتاست با موفقیت بارگذاری شد. تعداد رکوردها: {len(df)}")
            return df
        except Exception as e:
            print(f"خطا در بارگذاری دیتاست: {e}")
            return None

    def preprocess_text(self, text):
        """پیش‌پردازش متن فارسی"""
        if not isinstance(text, str):
            return ""

        # نرمال‌سازی متن
        text = self.normalizer.normalize(text)

        # حذف علائم نگارشی و اعداد
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\d+', ' ', text)

        # توکن‌سازی و ریشه‌یابی
        tokens = word_tokenize(text)
        tokens = [self.stemmer.stem(token) for token in tokens if token.strip() != '']

        return ' '.join(tokens)

    def prepare_dataset(self, df, text_column='text', label_column='label'):
        """آماده‌سازی دیتاست برای آموزش"""
        # پیش‌پردازش متون
        df['processed_text'] = df[text_column].apply(self.preprocess_text)

        # تبدیل برچسب‌ها به عدد
        df['label'] = df[label_column].map({'real': 0, 'fake': 1})

        return df

    def train_tfidf_model(self, df):
        """آموزش مدل با روش TF-IDF"""
        # تقسیم داده به آموزش و آزمون
        X_train, X_test, y_train, y_test = train_test_split(
            df['processed_text'], df['label'], test_size=0.2, random_state=42)

        # تبدیل متن به بردار با TF-IDF
        self.vectorizer = TfidfVectorizer(max_features=5000)
        X_train_tfidf = self.vectorizer.fit_transform(X_train)
        X_test_tfidf = self.vectorizer.transform(X_test)

        # آموزش مدل رگرسیون لجستیک
        self.model = LogisticRegression(max_iter=1000)
        self.model.fit(X_train_tfidf, y_train)

        # ارزیابی مدل
        y_pred = self.model.predict(X_test_tfidf)
        print("نتایج ارزیابی مدل TF-IDF:")
        print(classification_report(y_test, y_pred))
        print(f"دقت مدل: {accuracy_score(y_test, y_pred):.2f}")

    def train_word2vec_model(self, df):
        """آموزش مدل Word2Vec برای متن‌های فارسی"""
        sentences = [word_tokenize(text) for text in df['processed_text']]
        self.word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

        # تبدیل متون به بردار با میانگین بردارهای کلمات
        def text_to_vector(text):
            words = word_tokenize(text)
            word_vectors = [self.word2vec_model.wv[word] for word in words if word in self.word2vec_model.wv]
            if len(word_vectors) == 0:
                return np.zeros(100)
            return np.mean(word_vectors, axis=0)

        df['word2vec'] = df['processed_text'].apply(text_to_vector)

        # تقسیم داده به آموزش و آزمون
        X = np.stack(df['word2vec'].values)
        y = df['label'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # آموزش مدل
        self.model = LogisticRegression(max_iter=1000)
        self.model.fit(X_train, y_train)

        # ارزیابی مدل
        y_pred = self.model.predict(X_test)
        print("\nنتایج ارزیابی مدل Word2Vec:")
        print(classification_report(y_test, y_pred))
        print(f"دقت مدل: {accuracy_score(y_test, y_pred):.2f}")

    def predict_news(self, news_text):
        """پیش‌بینی واقعی یا جعلی بودن خبر"""
        if self.model is None or self.vectorizer is None:
            print("لطفا ابتدا مدل را آموزش دهید")
            return None

        # پیش‌پردازش متن ورودی
        processed_text = self.preprocess_text(news_text)

        if self.word2vec_model:
            # اگر از Word2Vec استفاده شده
            vector = np.mean([self.word2vec_model.wv[word] for word in word_tokenize(processed_text)
                            if word in self.word2vec_model.wv] or [np.zeros(100)], axis=0)
            vector = vector.reshape(1, -1)
        else:
            # اگر از TF-IDF استفاده شده
            vector = self.vectorizer.transform([processed_text])

        # پیش‌بینی
        prediction = self.model.predict(vector)[0]
        probability = self.model.predict_proba(vector)[0]

        result = {
            'text': news_text,
            'processed_text': processed_text,
            'prediction': 'جعلی' if prediction == 1 else 'واقعی',
            'fake_probability': f"{probability[1]:.2%}",
            'real_probability': f"{probability[0]:.2%}"
        }

        return result

    def display_result(self, result):
        """نمایش زیبای نتیجه"""
        if not result:
            return

        reshaped_text = arabic_reshaper.reshape(result['text'])
        bidi_text = get_display(reshaped_text)

        print("\n" + "="*50)
        print(f"متن اصلی:\n{bidi_text}")
        print("\n" + "-"*50)
        print(f"متن پردازش شده:\n{result['processed_text']}")
        print("\n" + "-"*50)
        print(f"نتیجه تشخیص: {get_display(arabic_reshaper.reshape(result['prediction']))}")
        print(f"احتمال جعلی بودن: {result['fake_probability']}")
        print(f"احتمال واقعی بودن: {result['real_probability']}")
        print("="*50 + "\n")

ModuleNotFoundError: No module named 'arabic_reshaper'